### Get Azure Machine Learning Workspace

Workspace is used to retreive credentials and secrets in Key Vault to connect to Azure Data Explorer.

In [ ]:
from obs.drift import Drift_Analysis
from azureml.core import Workspace
from obs.drift import launch_dashboard, execute

ws = Workspace.from_config()
drift_analysis = Drift_Analysis(ws)

### Option 1: Local Dashboard with Dash

Run cell below and copy the link into browser to access the dashboard. Once in the dashboard, interact with the widgets to create visualizations from avaiable data. Use ```irisdata``` created in part 0 as a sample table if needed. 

In [ ]:
launch_dashboard(drift_analysis)

### Option 2: Launch drift detection job and use  Dashboard to Azure Data Explorer to view result



1. Launch data detection job which is an Azure ML job to compute drift 

In [ ]:
from obs.drift.drift_analysis_scheduler import execute
from datetime import datetime
from azure.ai.ml.constants import TimeZone
from dateutil import tz

from azure.ai.ml.entities import (
    CronSchedule,

    ScheduleStatus,
)
subscription_id= "0e9bace8-7a81-4922-83b5-d995ff706507"
resource_group = "azureml"
workspace = "ws01ent"
compute_name = "cpu-cluster"
base_table_name = "IRIS_DATA_NEW"
target_table_name = base_table_name
base_dt_from = "12/15/2021"
base_dt_to = "03/01/2022"
target_dt_from = "04/01/2022"
target_dt_to ="05/13/2022"

schedule_start_time = datetime.now(tz=tz.gettz("PACIFIC STANDARD TIME"))
cron_schedule = CronSchedule(
    expression="*/10 * * * *",
    start_time=schedule_start_time,
    time_zone=TimeZone.PACIFIC_STANDARD_TIME,
    status=ScheduleStatus.ENABLED,
)

ml_client, job_name = execute(subscription_id=subscription_id,resource_group=resource_group,workspace=workspace, compute_name =compute_name, 
base_table_name =base_table_name,target_table_name =target_table_name, base_dt_from =base_dt_from, base_dt_to= base_dt_to,target_dt_from=target_dt_from, 
target_dt_to=target_dt_to, bin="7d", limit=100000, concurrent_run=False, drift_threshold =0.4)
#If you need to provide your own data drift logic, please supply the module using cron_schedule =None,user_defined_module_file="drift_job/drift_analysis_user.py", user_defined_conda_file="drift_job/conda.yml", 
# drift_analysis_job_file="drift_job/drift_analysis_aml_job.py",
#check the example of user defined modules in aml-obs-client/obs/test/drift_job


 2.Go to Azure Data Explorer Dashboards and import the quick_start/ADX_dashboards/databoard_drift_detection.json file to create a dashboard.

Review the instructions on how to import the dashboard [here.](https://docs.microsoft.com/en-us/azure/data-explorer/azure-data-explorer-dashboards#to-create-new-dashboard-from-a-file)

Default parameters may need to updated to ensure proper connection.